## Import Libraries

In [4]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

## Collect Data From ZRankings.com

In [ ]:
page = 0
result = []

# Loop through 7 pages of snow resort data
while page < 8:
    page = page + 1
    
    # Request to extract data from zrankings.com
    response = requests.get('https://www.zrankings.com/ski-resorts/snow?_=1615734995765&page=' + str(page))
    text = BeautifulSoup(response.text, 'html.parser')
    
    # Div tag where all data we want exists
    snowTable = text.find('div', {'class': 'tableshow table-wrap'})
    rows = snowTable.find_all('tr')
    
    # Iterating through each row of ski resorts and striping text from tags
    for row in rows[1:]:
        td = row.find_all('td')
        snowData = {}
        snowData['Zone'] = td[2].text.strip()
        resortName = td[1].text.strip()
        snowData['Resort Name'] = resortName.replace("\n","")[:-2]
        snowData['State'] = resortName.replace("\n","")[-2:]
        snowData['Base Elevation (ft)'] = re.findall('\d+', td[6].text.strip())[0]
        snowData['Top Elevation (ft)'] = re.findall('\d+', td[6].text.strip())[1]
        snowData['Snowfall Score'] = td[11].text.strip()
        snowData['% of North-facing Terrain'] = td[12].text.strip().replace('%', '')
        snowData['% of South-facing Terrain'] = td[15].text.strip().replace('%', '')
        snowData['% of East-facing Terrain'] = td[13].text.strip().replace('%', '')
        snowData['% of West-facing Terrain'] = td[14].text.strip().replace('%', '')
        snowData['Total Snow Score With Preservation'] = td[16].text.strip()
        
        # Parse ski resort link to access more information on that resort
        moreLink = td[17].a["href"]
        response = requests.get('https://www.zrankings.com' + moreLink)
        text = BeautifulSoup(response.text, 'html.parser')
        overallScore = text.find_all('div', {'class': 'overall-compare'})
        
        '''
        Collect further information within a ski resort's provided direct link
        Try/Exceptions are used in the case that a tag does not exist on the page.
        '''
        
        for score in overallScore:
            div1 = score.find_all('div')
            try:
                snowData['Overall Ranking'] =  re.sub(r"[\n\t\s]*", "", div1[3].find('h4').text)  
            except:
                continue
            try:
                snowData['Regional Ranking'] = div1[6].find('h4').text # Rank in region
            except:
                continue
            try:
                snowData['State Ranking'] = div1[8].find('h4').text # Rank in state
            except:
                continue
                
        snowStuff = text.find_all('div', {'class': 'snow-stuff'})
        for stuff in snowStuff:
            div2 = stuff.find_all('div')
            try:
                snowData['True Snow Per Year (inches)'] = div2[1]('h3')[0].text[:-1] # True Snow Per Year
            except:
                continue
            try:
                snowData['Snow Quality Rank'] = div2[3]('h2')[0].text # Snow quality rank
            except:
                continue
            try:
                snowData['Dump Potential Rank'] = div2[6]('h2')[0].text # Dump Potential Rank
            except:
                continue
            try:
                snowData['% of Days With More Than 6" of Snow'] = div2[11]('span')[0].text.replace('%', '') #% of days with more than 6" of snow
            except:
                continue
            try:
                snowData['% of Months With More Than 90" of Snow'] = div2[15]('span')[0].text.replace('%', '') # % of months with more than 90" of snow
            except:
                continue
            try:
                snowData['% of Months With Less Than 30" of Snow'] = div2[19]('span')[0].text.replace('%', '') # % of months with less than 30" of snow
            except:
                continue
 
        sideStats = text.find_all('div', {'class': 'side-stats-2 clearfix'})
        for stats in sideStats:
            span = stats.find_all('span')
            snowData['Acreage'] = span[0].text.replace("acres", "").replace(',', '')
            snowData['Total Runs'] = span[1].text
            snowData['Longest Run (ft)'] = span[2].text.replace('ft', '').replace(',', '')
            snowData['Lifts'] = span[3].text
            snowData['Uphill Max (ppl/hr)'] = span[4].text.replace('ppl/hr', '').replace(',', '')
            snowData['Terrain Parks'] = span[5].text
            snowData['Halfpipes'] = span[7].text
        
        #snowData['Total Snow Score With Preservation'] = td[17].a["href"]
        result.append(snowData)
    print(page)

1
2


## Construct Data to Pandas Dataframe

In [1]:
snowData = pd.DataFrame.from_dict(result)
pd.set_option('display.max_columns', 35)
snowData.head()

NameError: name 'pd' is not defined

## Convert to CSV

In [20]:
snowData.to_csv('zRankings.csv')